In [1]:
from sympy.physics.quantum.qubit import Qubit
from sympy import Matrix, sqrt, symbols, simplify
import sympy as sp
from IPython.display import display, Latex
from itertools import product

In [2]:
from sympy.physics.quantum import Ket, Bra, Dagger, represent, TensorProduct

In [3]:
from sympy.physics.matrices import msigma

In [4]:
import random
import math
import cmath

In [5]:
import numpy as np

In [6]:
import itertools

In [7]:
# Step 1: Symbols define karein
alpha, beta = symbols('α β')

# Step 2: Quantum state banayein (example: superposition state)
psi = alpha * Qubit('0') + beta * Qubit('1')

# Step 3: Density matrix calculate karein
rho = psi * Dagger(psi)

# Step 4: Matrix ko expand aur simplify karein
density_matrix = rho.doit().expand()
simplified_matrix = simplify(density_matrix)
simplified_matrix

α*Dagger(α)*|0>*<0| + α*Dagger(β)*|0>*<1| + β*Dagger(α)*|1>*<0| + β*Dagger(β)*|1>*<1|

In [8]:
res = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
res

# this is for one qubit, lets do for 2

Matrix([
[α*Dagger(α), α*Dagger(β)],
[β*Dagger(α), β*Dagger(β)]])

In [7]:
def get_rho_bipartite(alpha, beta, gamma, delta):
    psi = alpha * TensorProduct(Qubit(0), Qubit(0)) + beta * TensorProduct(Qubit(0), Qubit(1)) + gamma * TensorProduct(Qubit(1), Qubit(0)) + delta * TensorProduct(Qubit(1), Qubit(1))
    
    rho = psi * Dagger(psi)
    density_matrix = rho.doit().expand()
    simplified_matrix = simplify(density_matrix)
    print(simplified_matrix)
    
    rho = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
    return rho

In [8]:
dA, dB = 2, 2

In [9]:
def permute_dims(tensor, perm):
    """
    Permutes the dimensions of a sympy MutableDenseNDimArray according to the given permutation tuple.
    """
    old_shape = tensor.shape
    new_shape = tuple(old_shape[i] for i in perm)
    new_tensor = sp.MutableDenseNDimArray.zeros(*new_shape)
    for new_index in product(*[range(s) for s in new_shape]):
        # Compute the corresponding old index.
        old_index = [None] * len(new_index)
        for new_axis, old_axis in enumerate(perm):
            old_index[old_axis] = new_index[new_axis]
        new_tensor[new_index] = tensor[tuple(old_index)]
    return new_tensor


In [10]:
def compute_entanglement_measures_symbolic(rho, dA, dB, display_output=True):
    """
    Computes entanglement measures for a bipartite system (dA x dB).
    """
    N = dA * dB
    assert rho.shape[0] == N, "Product dA*dB must equal the matrix dimension."
    
    if display_output:
        display(Latex(r"Density matrix $\rho$: "))
        display(Latex(r"$$" + sp.latex(rho) + r"$$"))
    
    eig_dict_input = rho.eigenvals()
    eigenvals_input = [sp.simplify(ev) for ev, mult in eig_dict_input.items() for _ in range(mult)]
    if display_output:
        display(Latex(r"Eigenvalues of $\rho$: "))
        display(Latex(r"$$" + sp.latex(eigenvals_input) + r"$$"))
    
    # Partial Transpose
    rho_tensor = sp.MutableDenseNDimArray(rho, (dA, dB, dA, dB))
    rho_pt_tensor = permute_dims(rho_tensor, (0, 3, 2, 1))
    rho_pt = sp.Matrix(rho_pt_tensor.reshape(N, N))
    if display_output:
        display(Latex(r"Partial transpose $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(rho_pt) + r"$$"))
    
    eig_dict_pt = rho_pt.eigenvals()
    eigenvals_pt = [sp.simplify(ev) for ev, mult in eig_dict_pt.items() for _ in range(mult)]
    if display_output:
        display(Latex(r"Eigenvalues of $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(eigenvals_pt) + r"$$"))
    
    det_rho_pt = sp.simplify(rho_pt.det())
    if display_output:
        display(Latex(r"Determinant of $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(det_rho_pt) + r"$$"))
    
    # Realignment
    rho_realigned_tensor = permute_dims(rho_tensor, (0, 2, 1, 3))
    realigned = sp.Matrix(rho_realigned_tensor.reshape(dA*dA, dB*dB))
    if display_output:
        display(Latex(r"Realigned matrix: "))
        display(Latex(r"$$" + sp.latex(realigned) + r"$$"))
    
    rhotilde_t_rhotilde = realigned.T * realigned
    if display_output:
        display(Latex(r"Symmetric for realigned: "))
        display(Latex(r"$$" + sp.latex(rhotilde_t_rhotilde) + r"$$"))
    
    # sum_sv = sp.simplify(sum(singular_vals))
    # if display_output:
    #     display(Latex(r"Sum of singular values: "))
    #     display(Latex(r"$$" + sp.latex(sum_sv) + r"$$"))
    
    # product_sv = sp.simplify(sp.Mul(*singular_vals))
    # if display_output:
    #     display(Latex(r"Product of singular values: "))
    #     display(Latex(r"$$" + sp.latex(product_sv) + r"$$"))
    
    # det_realigned = sp.simplify(realigned.det())
    # if display_output:
    #     display(Latex(r"Determinant of the realigned matrix: "))
    #     display(Latex(r"$$" + sp.latex(det_realigned) + r"$$"))
    
    # product_eigenvals = sp.simplify(sp.Mul(*eigenvals_pt))
    # if display_output:
    #     display(Latex(r"Product of eigenvalues of $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(product_eigenvals) + r"$$"))
    
    #return eigenvals_input, eigenvals_pt, singular_vals, sum_sv, product_sv, product_eigenvals, det_rho_pt, det_realigned
    return realigned, rhotilde_t_rhotilde


In [11]:
alpha, beta, gamma, delta= symbols('α β γ δ')
realigned, rhotilde_t_rhotilde= compute_entanglement_measures_symbolic(get_rho_bipartite(alpha, beta, gamma, delta), dA, dB, True)

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + α*Dagger(γ)*|0>x|0>*<1|x<0| + α*Dagger(δ)*|0>x|0>*<1|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1| + β*Dagger(γ)*|0>x|1>*<1|x<0| + β*Dagger(δ)*|0>x|1>*<1|x<1| + γ*Dagger(α)*|1>x|0>*<0|x<0| + γ*Dagger(β)*|1>x|0>*<0|x<1| + γ*Dagger(γ)*|1>x|0>*<1|x<0| + γ*Dagger(δ)*|1>x|0>*<1|x<1| + δ*Dagger(α)*|1>x|1>*<0|x<0| + δ*Dagger(β)*|1>x|1>*<0|x<1| + δ*Dagger(γ)*|1>x|1>*<1|x<0| + δ*Dagger(δ)*|1>x|1>*<1|x<1|


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [12]:
realigned

Matrix([
[α*Dagger(α), α*Dagger(β), β*Dagger(α), β*Dagger(β)],
[α*Dagger(γ), α*Dagger(δ), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), δ*Dagger(α), δ*Dagger(β)],
[γ*Dagger(γ), γ*Dagger(δ), δ*Dagger(γ), δ*Dagger(δ)]])

In [13]:
rhotilde_t_rhotilde

Matrix([
[                            α**2*Dagger(α)**2 + α**2*Dagger(γ)**2 + γ**2*Dagger(α)**2 + γ**2*Dagger(γ)**2, α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ)],
[α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                             α**2*Dagger(β)**2 + α**2*Dagger(δ)**2 + γ**2*Dagger(β)**2 + γ**2*Dagger(δ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(β)**2 + α*β*Dagger(δ)**2 + γ*δ*Dagger(β)**2 + γ*δ*Dagger(δ)**2],
[                                α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagge

In [14]:
def generate_alpha_beta_gamma_delta():
    # Generate four random complex numbers using a Gaussian distribution.
    # Using a Gaussian can help ensure a uniform distribution on the complex sphere.
    a = random.gauss(0,1) + 1j * random.gauss(0,1)
    b = random.gauss(0,1) + 1j * random.gauss(0,1)
    c = random.gauss(0,1) + 1j * random.gauss(0,1)
    d = random.gauss(0,1) + 1j * random.gauss(0,1)
    
    # Normalize the state so that |α|^2 + |β|^2 + |γ|^2 + |δ|^2 = 1
    norm = math.sqrt(abs(a)**2 + abs(b)**2 + abs(c)**2 + abs(d)**2)
    alpha = a / norm
    beta  = b / norm
    gamma = c / norm
    delta = d / norm

    return alpha, beta, gamma, delta

In [15]:
a, b, c, d= generate_alpha_beta_gamma_delta()

In [16]:
rhotilde_t_rhotilde_subbed= simplify(rhotilde_t_rhotilde.subs({alpha: a, beta: b, gamma: c, delta: d}))

In [17]:
eig_dict= rhotilde_t_rhotilde_subbed.eigenvals()
eig_dict

{-0.00881549268689383 - 0.0649960631459152*I: 1,
 0.0141397088499269 + 1.92296744548659e-34*I: 1,
 -0.00881549268689383 + 0.0649960631459152*I: 1,
 0.304263771018364 - 9.5789231331853e-35*I: 1}

In [18]:
def get_singular_values_from_eig_dict(eig_dict):
    singular_vals= []
    for eigenvalue, multiplicity in eig_dict.items():
        # Simplify the eigenvalue for a cleaner expression.
        eigenvalue = sp.simplify(eigenvalue)
        # print("current eig: ", eigenvalue)
        # print("multiplicity: ", multiplicity)
        # Assume eigenvalues are nonnegative; take the square root.
        s_val = sp.sqrt(eigenvalue).evalf()
        singular_vals.extend([sp.simplify(s_val)] * multiplicity)
    return singular_vals

In [19]:
singular_vals= get_singular_values_from_eig_dict(eig_dict)
singular_vals

[0.168486904404796 - 0.192881646723593*I,
 0.118910507735552 + 0.e-33*I,
 0.168486904404796 + 0.192881646723593*I,
 0.55160109773129 - 0.e-34*I]

In [20]:
sum(singular_vals)

1.00748541427643 - 0.e-34*I

In [21]:
def generate_1_normalized_real(n):
    # Generate n random real numbers from a Gaussian distribution
    random_numbers= [random.gauss(0, 1) for i in range(n)]
    
    # Compute the norm: sqrt(a^2 + b^2 + c^2 + d^2)
    sqsum= sum(map(lambda x: x**2, random_numbers))
    norm= sqrt(sqsum)
    return tuple(map(lambda x: x / norm, random_numbers))

In [22]:
rho= get_rho_bipartite(alpha, beta, gamma, delta)
rho

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + α*Dagger(γ)*|0>x|0>*<1|x<0| + α*Dagger(δ)*|0>x|0>*<1|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1| + β*Dagger(γ)*|0>x|1>*<1|x<0| + β*Dagger(δ)*|0>x|1>*<1|x<1| + γ*Dagger(α)*|1>x|0>*<0|x<0| + γ*Dagger(β)*|1>x|0>*<0|x<1| + γ*Dagger(γ)*|1>x|0>*<1|x<0| + γ*Dagger(δ)*|1>x|0>*<1|x<1| + δ*Dagger(α)*|1>x|1>*<0|x<0| + δ*Dagger(β)*|1>x|1>*<0|x<1| + δ*Dagger(γ)*|1>x|1>*<1|x<0| + δ*Dagger(δ)*|1>x|1>*<1|x<1|


Matrix([
[α*Dagger(α), α*Dagger(β), α*Dagger(γ), α*Dagger(δ)],
[β*Dagger(α), β*Dagger(β), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), γ*Dagger(γ), γ*Dagger(δ)],
[δ*Dagger(α), δ*Dagger(β), δ*Dagger(γ), δ*Dagger(δ)]])

In [23]:
def get_chsh_score(t_matrix, subsdict= None):
    no_of_ss_to_be_considered= 2
    
    if subsdict is not None:
        t_matrix= t_matrix.subs(subsdict)
        
    t_symm= t_matrix.T * t_matrix
    ssquared_dict= t_symm.eigenvals()
    print("t symm eigs: ", ssquared_dict)
    ssquared_list= []
    for ssquared, multiplicity in ssquared_dict.items():
        ssquared_list.extend([sp.simplify(ssquared)] * multiplicity)

    if subsdict is not None:
        greatest= sorted(ssquared_list, reverse=True)[:no_of_ss_to_be_considered]
    else:
        greatest= ssquared_list
    return 2* sqrt(sum(greatest))

In [24]:
def compare_realignment_scores_with_bchsh(rhotilde_t_rhotilde, t_matrix, alpha, beta, gamma, delta, no_of_random_samples= 20):
    res= {}
    for i in range(no_of_random_samples):
        a, b, c, d= generate_1_normalized_real(4)
        subsdict= {alpha: a, beta: b, gamma: c, delta: d}
        subbed_symm= simplify(rhotilde_t_rhotilde.subs(subsdict))
        try:
            eig_dict= subbed_symm.eigenvals()
            print("eig_dict of sub symm: ", eig_dict)
            chsh_score= get_chsh_score(t_matrix, subsdict)
            print("chsh score: ", chsh_score)
            res[str((a, b, c, d))]= [sum(get_singular_values_from_eig_dict(eig_dict)), chsh_score]
        except e:
            print(e)
    return res
    

In [25]:
realigned, rhotilde_t_rhotilde= compute_entanglement_measures_symbolic(rho, dA, dB, False)
realigned

Matrix([
[α*Dagger(α), α*Dagger(β), β*Dagger(α), β*Dagger(β)],
[α*Dagger(γ), α*Dagger(δ), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), δ*Dagger(α), δ*Dagger(β)],
[γ*Dagger(γ), γ*Dagger(δ), δ*Dagger(γ), δ*Dagger(δ)]])

In [26]:
rhotilde_t_rhotilde

Matrix([
[                            α**2*Dagger(α)**2 + α**2*Dagger(γ)**2 + γ**2*Dagger(α)**2 + γ**2*Dagger(γ)**2, α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ)],
[α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                             α**2*Dagger(β)**2 + α**2*Dagger(δ)**2 + γ**2*Dagger(β)**2 + γ**2*Dagger(δ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(β)**2 + α*β*Dagger(δ)**2 + γ*δ*Dagger(β)**2 + γ*δ*Dagger(δ)**2],
[                                α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagge

In [27]:
t_nm = lambda n, m, rho: (rho * TensorProduct(msigma(n), msigma(m))).trace()
make_t_matrix= lambda rho: Matrix([[t_nm(n, m, rho) for m in range(1, 4)] for n in range(1, 4)])

In [28]:
t_matrix= make_t_matrix(rho)
t_matrix

Matrix([
[        α*Dagger(δ) + β*Dagger(γ) + γ*Dagger(β) + δ*Dagger(α), I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α),         α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β)],
[I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α),        -α*Dagger(δ) + β*Dagger(γ) + γ*Dagger(β) - δ*Dagger(α), I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)],
[        α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ), I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ),         α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ)]])

In [29]:
compare_realignment_scores_with_bchsh(rhotilde_t_rhotilde, t_matrix, alpha, beta, gamma, delta)

eig_dict of sub symm:  {0.00110185502846857: 1, 0.997795073175677: 1, 0.00110185502846853: 1, 1.21676738679518e-6: 1}
t symm eigs:  {0.00440742011387422: 1, 1.00000000000000: 1, 0.00440742011387426: 1}
chsh score:  2.00440257444843
eig_dict of sub symm:  {0.00328733719477666: 1, 1.08782249549554e-5: 1, 0.00328733719477666: 1, 0.993414447385492: 1}
t symm eigs:  {0.0131493487791066: 1, 0.0131493487791066: 1, 1.00000000000000: 1}
chsh score:  2.01310640432055
eig_dict of sub symm:  {0.970943410064681: 1, 0.000214194708414963: 1, 0.0144211976134517: 1, 0.0144211976134517: 1}
t symm eigs:  {0.0576847904538070: 2, 1.00000000000000: 1}
chsh score:  2.05687606865733
eig_dict of sub symm:  {0.156779958550828: 2, 0.648539614108988: 1, 0.0379004687893569: 1}
t symm eigs:  {0.627119834203311: 1, 0.627119834203311: 1, 1.00000000000000: 1}
chsh score:  2.55117214958404
eig_dict of sub symm:  {0.0153504587925342: 1, 0.000243160977533445: 1, 0.0153504587925342: 1, 0.969055921437398: 1}
t symm eigs:  

{'(0.802693422062294, 0.331881140384836, -0.441955308496818, -0.224084100565539)': [1.06638840345929,
  2.00440257444843],
 '(0.0521698217683261, 0.0667945974245705, -0.0828383108857986, 0.992952469003999)': [1.11467060991861,
  2.01310640432055],
 '(0.568705590355028, -0.807131464441798, -0.0409109497496003, -0.153098154128529)': [1.24017658181806,
  2.05687606865733],
 '(0.0134973806302668, -0.443428495086048, 0.889642856095188, 0.108280095403727)': [1.79190898100938,
  2.55117214958404],
 '(-0.134649380618047, 0.0902270338206709, -0.0928970915372632, 0.982394399948929)': [1.24779393691157,
  2.06048716100842],
 '(0.343062710988880, 0.0808296232108244, 0.812931636281501, 0.463591094687159)': [1.18666371975377,
  2.03454500493090],
 '(-0.311099113431246, -0.0208069078358944, -0.197870087437233, 0.929317944896247)': [1.58645410685151,
  2.31855853447180],
 '(-0.711739082585108, -0.270440814174114, -0.276650096771448, -0.586305354150975)': [1.68495791493187,
  2.42418427123667],
 '(0.59

In [ ]:
# Bell state (|00⟩ + |11⟩)/√2
bell_state = (TensorProduct(Qubit('0'), Qubit('0')) + 
             TensorProduct(Qubit('1'), Qubit('1')))/sqrt(2)

# Density matrix
bell_rho = bell_state * Dagger(bell_state)
bell_rho

In [ ]:
matrix_form = represent(bell_rho.doit(), basis_set=[Qubit('0'), Qubit('1')])

# Simplify करें और output को Matrix के रूप में प्राप्त करें
final_matrix = Matrix(simplify(matrix_form)).expand()
final_matrix

In [ ]:
# Bell state vector (|00⟩ + |11⟩)/√2 को Matrix के रूप में परिभाषित करें
bell_vector = Matrix([[1], [0], [0], [1]])/sqrt(2)

# Density matrix बनाएं (outer product)
density_matrix = bell_vector * Dagger(bell_vector)

# मैट्रिक्स को simplify करें और expand करें
rho = density_matrix.expand()
rho

In [ ]:
# Execute the optimized function

result = compute_entanglement_measures_symbolic(rho, dA, dB)
result

In [ ]:
ajeeb_matrix= sp.Matrix([
    [0.5, 0, 0, 0],
    [0, 0, 0.5, 0],
    [0, 0.5, 0, 0],
    [0, 0, 0, 0.5]
])
ajeeb_matrix


In [ ]:
ajeeb_matrix.det()

In [ ]:
1 / 16

# Making the Density matrix in Bell basis for bipartite system, to do realignment on it later

In [30]:
alpha, beta, gamma, delta= symbols('α β γ δ')
rho= get_rho_bipartite(alpha, beta, gamma, delta)

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + α*Dagger(γ)*|0>x|0>*<1|x<0| + α*Dagger(δ)*|0>x|0>*<1|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1| + β*Dagger(γ)*|0>x|1>*<1|x<0| + β*Dagger(δ)*|0>x|1>*<1|x<1| + γ*Dagger(α)*|1>x|0>*<0|x<0| + γ*Dagger(β)*|1>x|0>*<0|x<1| + γ*Dagger(γ)*|1>x|0>*<1|x<0| + γ*Dagger(δ)*|1>x|0>*<1|x<1| + δ*Dagger(α)*|1>x|1>*<0|x<0| + δ*Dagger(β)*|1>x|1>*<0|x<1| + δ*Dagger(γ)*|1>x|1>*<1|x<0| + δ*Dagger(δ)*|1>x|1>*<1|x<1|


In [31]:
I_2= Matrix.eye(2)
rx, ry, rz, sx, sy, sz = symbols('rx ry rz sx sy sz')
rho

Matrix([
[α*Dagger(α), α*Dagger(β), α*Dagger(γ), α*Dagger(δ)],
[β*Dagger(α), β*Dagger(β), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), γ*Dagger(γ), γ*Dagger(δ)],
[δ*Dagger(α), δ*Dagger(β), δ*Dagger(γ), δ*Dagger(δ)]])

In [32]:
rho.trace()

α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) + δ*Dagger(δ)

In [33]:
T= make_t_matrix(rho)
display(T)

Matrix([
[        α*Dagger(δ) + β*Dagger(γ) + γ*Dagger(β) + δ*Dagger(α), I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α),         α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β)],
[I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α),        -α*Dagger(δ) + β*Dagger(γ) + γ*Dagger(β) - δ*Dagger(α), I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)],
[        α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ), I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ),         α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ)]])

In [34]:
bchsh_sum_correlations= Matrix.zeros(4)
for n in range(1, 4):
    for m in range(1, 4):
        tba= T[n - 1, m - 1] * TensorProduct(msigma(n), msigma(m))
        bchsh_sum_correlations += tba
    # print("sum at the end of n= ", n, "is: ", s)

In [35]:
bchsh_sum_correlations

Matrix([
[                                                                                                                α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ),                                             α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) - I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                             α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)), 2*α*Dagger(δ) + 2*δ*Dagger(α) - I*(I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α)) - I*(I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α))],
[                                            α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) + I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                                                                                                -α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) -

In [36]:
s= Matrix([sx, sy, sz])
r= Matrix([rx, ry, rz])

In [37]:
ssigma= Matrix.zeros(2)
rsigma= Matrix.zeros(2)
for i in range(1, 4):
    ssigma += s[i-1] * msigma(i)
    rsigma += r[i-1] * msigma(i)

In [38]:
rho_bell_basis= Matrix.zeros(4)

In [39]:
rho_bell_basis+= TensorProduct(I_2, I_2)
rho_bell_basis

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

In [40]:
res= TensorProduct(rsigma, I_2)
rho_bell_basis += res
res

Matrix([
[       rz,         0, rx - I*ry,         0],
[        0,        rz,         0, rx - I*ry],
[rx + I*ry,         0,       -rz,         0],
[        0, rx + I*ry,         0,       -rz]])

In [41]:
res=  TensorProduct(I_2, ssigma)
rho_bell_basis += res
res

Matrix([
[       sz, sx - I*sy,         0,         0],
[sx + I*sy,       -sz,         0,         0],
[        0,         0,        sz, sx - I*sy],
[        0,         0, sx + I*sy,       -sz]])

In [42]:
rho_bell_basis += bchsh_sum_correlations
bchsh_sum_correlations

Matrix([
[                                                                                                                α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ),                                             α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) - I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                             α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)), 2*α*Dagger(δ) + 2*δ*Dagger(α) - I*(I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α)) - I*(I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α))],
[                                            α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) + I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                                                                                                -α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) -

In [43]:
rho_bell_basis

Matrix([
[                                                                                                  rz + sz + α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ) + 1,                                 sx - I*sy + α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) - I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                 rx - I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)), 2*α*Dagger(δ) + 2*δ*Dagger(α) - I*(I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α)) - I*(I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α))],
[                                sx + I*sy + α*Dagger(β) + β*Dagger(α) - γ*Dagger(δ) - δ*Dagger(γ) + I*(I*α*Dagger(β) - I*β*Dagger(α) - I*γ*Dagger(δ) + I*δ*Dagger(γ)),                                                                                                   rz - sz - α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) - δ*D

In [44]:
rho_tensor = sp.MutableDenseNDimArray(rho_bell_basis, (dA, dB, dA, dB))
rho_realigned_tensor = permute_dims(rho_tensor, (0, 2, 1, 3))
realigned = sp.Matrix(rho_realigned_tensor.reshape(dA*dA, dB*dB))
rhotilde_t_rhotilde= realigned.T * realigned
rhotilde_t_rhotilde

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    (rx - I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)))**2 + (rx + I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) + I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)))**2 + (-rz + sz - α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) - δ*Dagger(δ) + 1)**2 + (rz + sz + α*Dagger(α) - β*Dagger(β)

In [45]:
print(rhotilde_t_rhotilde)

Matrix([[(rx - I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)))**2 + (rx + I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) + I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β)))**2 + (-rz + sz - α*Dagger(α) + β*Dagger(β) + γ*Dagger(γ) - δ*Dagger(δ) + 1)**2 + (rz + sz + α*Dagger(α) - β*Dagger(β) - γ*Dagger(γ) + δ*Dagger(δ) + 1)**2, (2*α*Dagger(δ) + 2*δ*Dagger(α) - I*(I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α)) - I*(I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α)))*(rx - I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) - I*(I*α*Dagger(γ) - I*β*Dagger(δ) - I*γ*Dagger(α) + I*δ*Dagger(β))) + (2*β*Dagger(γ) + 2*γ*Dagger(β) - I*(I*α*Dagger(δ) - I*β*Dagger(γ) + I*γ*Dagger(β) - I*δ*Dagger(α)) + I*(I*α*Dagger(δ) + I*β*Dagger(γ) - I*γ*Dagger(β) - I*δ*Dagger(α)))*(rx + I*ry + α*Dagger(γ) - β*Dagger(δ) + γ*Dagger(α) - δ*Dagger(β) + I*(I*α*Dagger(γ)

# now find the Bchsh condition for this general rho matrix

In [46]:
# if no substitution is done, all the singular values of T are considered instead of just two
# because you cant compare expressions? can you?
chsh_score= get_chsh_score(T)
chsh_score

t symm eigs:  {4*α*δ*Dagger(α)*Dagger(δ) - 4*α*δ*Dagger(β)*Dagger(γ) - 4*β*γ*Dagger(α)*Dagger(δ) + 4*β*γ*Dagger(β)*Dagger(γ): 2, α**2*Dagger(α)**2 + 2*α*β*Dagger(α)*Dagger(β) + 2*α*γ*Dagger(α)*Dagger(γ) + 2*α*δ*Dagger(α)*Dagger(δ) + β**2*Dagger(β)**2 + 2*β*γ*Dagger(β)*Dagger(γ) + 2*β*δ*Dagger(β)*Dagger(δ) + γ**2*Dagger(γ)**2 + 2*γ*δ*Dagger(γ)*Dagger(δ) + δ**2*Dagger(δ)**2: 1}


2*sqrt(α**2*Dagger(α)**2 + 2*α*β*Dagger(α)*Dagger(β) + 2*α*γ*Dagger(α)*Dagger(γ) + 10*α*δ*Dagger(α)*Dagger(δ) - 8*α*δ*Dagger(β)*Dagger(γ) + β**2*Dagger(β)**2 - 8*β*γ*Dagger(α)*Dagger(δ) + 10*β*γ*Dagger(β)*Dagger(γ) + 2*β*δ*Dagger(β)*Dagger(δ) + γ**2*Dagger(γ)**2 + 2*γ*δ*Dagger(γ)*Dagger(δ) + δ**2*Dagger(δ)**2)

## we now have a general expression of singular values, sum of the biggest 2 can be found after analysis, and thus comparison can be done between this rule and realigned general rho

In [47]:
generate_1_normalized_real(10)

(0.137233184586898,
 0.00712529277768793,
 0.550525181425600,
 0.0191287238588070,
 0.118255380554405,
 -0.695599836613160,
 -0.0408381571867490,
 -0.400650031830459,
 -0.121548405239709,
 -0.0535417048665619)

In [48]:
def generate_realignment_scores_bell_basis(rhotilde_t_rhotilde, rx, ry, rz, sx, sy, sz, no_of_random_samples= 20):
    res= {}
    for i in range(no_of_random_samples):
        a, b, c, d, e, f, g, h, i, j = generate_1_normalized_real(10)
        subbed_symm= simplify(rhotilde_t_rhotilde.subs({rx: a, ry: b, rz: c, sx: d, sy: e, sz: f, alpha: g, beta: h, gamma: i, delta: j}))
        print("subbed symm: ", subbed_symm)
        try:
            eig_dict= subbed_symm.eigenvals()
            res[str((a, b, c, d))]= sum(get_singular_values_from_eig_dict(eig_dict))
        except e:
            print(e)
    return res

In [49]:
generate_realignment_scores_bell_basis(rhotilde_t_rhotilde, rx, ry, rz, sx, sy, sz)

subbed symm:  Matrix([[3.27392821567761, 0.793535021198039 - 0.987553037392442*I, 0.793535021198039 + 0.987553037392442*I, 1.67877061887495], [0.793535021198039 - 0.987553037392442*I, 0.27974291194915 - 0.344077304194936*I, 0.653636969950706, 0.221880334344653 - 0.559658367703514*I], [0.793535021198039 + 0.987553037392442*I, 0.653636969950706, 0.27974291194915 + 0.344077304194936*I, 0.221880334344653 + 0.559658367703514*I], [1.67877061887495, 0.221880334344653 - 0.559658367703514*I, 0.221880334344653 + 0.559658367703514*I, 2.17630758681191]])
subbed symm:  Matrix([[2.14645288575775, -0.0657349610440739 + 0.331692154503324*I, -0.0657349610440739 - 0.331692154503324*I, 0.770140280692768], [-0.0657349610440739 + 0.331692154503324*I, 0.485014732534829 - 0.00885712906166704*I, 0.192851812068618, -0.61371006972304 + 0.325006035997477*I], [-0.0657349610440739 - 0.331692154503324*I, 0.192851812068618, 0.485014732534829 + 0.00885712906166704*I, -0.61371006972304 - 0.325006035997477*I], [0.77014

{'(0.359395643980454, -0.176848849134892, 0.0555195151107999, 0.193886082209290)': 4.26376447844523 + 0.e-33*I,
 '(0.0765007475985316, -0.539849387196041, -0.158803224308983, -0.123337500108516)': 4.21174954012086 + 0.e-32*I,
 '(0.593199874350130, 0.110154308118964, -0.280695152823607, 0.114178523709898)': 2.58346765556907 - 0.668042303947488*I,
 '(0.0736194357660213, -0.661185173473234, -0.327188789907657, -0.202781810579280)': 3.42067672826342 - 0.e-64*I,
 '(-0.194259344471788, 0.621890857296955, -0.223778268226467, 0.116708173288633)': 2.63527619404257 + 0.427875681316296*I,
 '(-0.172894011047486, -0.422320348518571, -0.0335851849800119, 0.131579205444338)': 2.87199976212909 - 0.e-64*I,
 '(-0.240602750934977, 0.171081009309855, -0.00952613918564805, -0.749712410482594)': 3.71203823381097 + 0.e-32*I,
 '(0.151062971909663, -0.241003698544867, 0.0656337247357289, -0.171747724574674)': 3.31722969332238 + 0.e-64*I,
 '(0.0599204129159702, -0.512473538444288, 0.234251047807275, 0.338027797

In [50]:
eigs= rhotilde_t_rhotilde.eigenvals()
eigs

KeyboardInterrupt: 